In [1]:
# Importing necessary libraries
import pandas as pd
pd.set_option('display.max_rows', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
#from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
from google.colab import files
uploaded = files.upload()

Saving survival_clean.csv to survival_clean.csv


In [4]:
import pandas as pd
df_full = pd.read_csv('survival_clean.csv')

In [5]:
df_full_categorical = ['apache_3j_bodysystem', 'apache_2_bodysystem', 'ethnicity',
                       'icu_admit_source', 'gender', 'icu_stay_type', 'icu_type' ]

df_full_binary = ['elective_surgery', 'arf_apache', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache',
                  'aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure', 'immunosuppression', 'leukemia',
                  'lymphoma', 'solid_tumor_with_metastasis', 'apache_post_operative']

df_full_multicat = ['gcs_eyes_apache', 'gcs_motor_apache', 'gcs_verbal_apache']

df_full_numeric = ['age', 'bmi', 'height', 'weight', 'pre_icu_los_days','heart_rate_apache', 'map_apache','resprate_apache',
                   'temp_apache', 'd1_potassium_min', 'd1_potassium_max', 'h1_mbp_max',
                   'h1_mbp_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_sysbp_max','h1_sysbp_min','d1_glucose_min',
                   'd1_glucose_max','h1_resprate_max','h1_resprate_min','h1_heartrate_max','h1_heartrate_min',
                   'd1_temp_max','d1_temp_min','d1_diasbp_max','d1_diasbp_min','d1_resprate_min','d1_resprate_max',
                   'd1_spo2_min','d1_spo2_max','h1_spo2_max','h1_spo2_min','d1_mbp_min','d1_mbp_max','d1_sysbp_min',
                   'd1_sysbp_max','d1_heartrate_max','d1_heartrate_min', 'apache_4a_hospital_death_prob',
                   'apache_4a_icu_death_prob']


In [6]:
# Assuming df is your DataFrame and 'target' is your target variable
X = df_full.drop('hospital_death', axis=1)  # Drop the target column
y = df_full['hospital_death']


In [8]:
!pip install -U imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 4.2 MB/s eta 0:00:00


In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_recall_fscore_support
#from xgboost import XGBClassifier
import pandas as pd

# Define categorical and numerical columns based on your dataset
categorical_cols = df_full_categorical
binary_cols = df_full_binary
multicat_cols = df_full_multicat
numeric_cols = df_full_numeric

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define preprocessing for categorical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols),
        ('multi', 'passthrough', multicat_cols),
        ('binary', 'passthrough', binary_cols),
        ('num', StandardScaler(), numeric_cols)
    ]
)

# Process X_train for SMOTE + Tomek Links
X_train_processed = preprocessor.fit_transform(X_train)

# Apply SMOTE with Tomek Links to balance classes
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train_processed, y_train)

# Check class distribution after resampling
print("Class distribution after SMOTE + Tomek Links:", pd.Series(y_resampled).value_counts())


Class distribution after SMOTE + Tomek Links: hospital_death
0    62821
1    62821
Name: count, dtype: int64


In [10]:
print("Shape of X_resampled (Training Data After SMOTE+Tomkek):", X_resampled.shape)
print("Number of rows in X_resampled:", X_resampled.shape[0])
print("Number of columns in X_resampled:", X_resampled.shape[1])

# Confirm the shape of the target variable after resampling
print("Number of rows in y_resampled:", len(y_resampled))

Shape of X_resampled (Training Data After SMOTE+Tomkek): (125642, 96)
Number of rows in X_resampled: 125642
Number of columns in X_resampled: 96
Number of rows in y_resampled: 125642


### Decision Trees

In [11]:
# Full Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the model on the resampled training data
clf.fit(X_resampled, y_resampled)

# Transform X_test with the same preprocessor (no SMOTE needed for test set)
X_test_processed = preprocessor.transform(X_test)

# Predict on the test set
y_pred = clf.predict(X_test_processed)

# Evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nTest Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nTrain Accuracy Score:", clf.score(X_train_processed, y_train))


Confusion Matrix:
 [[19348  1627]
 [ 1181   773]]

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93     20975
           1       0.32      0.40      0.36      1954

    accuracy                           0.88     22929
   macro avg       0.63      0.66      0.64     22929
weighted avg       0.89      0.88      0.88     22929


Test Accuracy Score: 0.8775349993458066

Train Accuracy Score: 0.9999854617352872


In [12]:
clf.get_depth()

48

In [13]:
# Import necessary libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Define parameter grid with refined ranges
dt_params = {
    'max_depth': range(10, 41, 3),               # Depths from 10 to 41 in increments of 3
    'min_samples_split': range(50, 201, 50),     # Min samples from 50 to 200 in increments of 50
    'max_leaf_nodes': range(20, 101, 20)         # Max leaf nodes from 20 to 100 in increments of 20
}

# Initialize Decision Tree Classifier with balanced class weights and consistent random state
opt_tree = DecisionTreeClassifier(random_state=42, class_weight='balanced')

# Set up GridSearchCV with 5-fold cross-validation, using accuracy as the scoring metric
grid_tree = GridSearchCV(
    estimator=opt_tree,
    param_grid=dt_params,
    cv=5,                       # 5-fold cross-validation
    scoring='f1_weighted',
    n_jobs=-1                    # Use all available cores for efficiency
)

# Fit the grid search on the resampled training data
grid_tree.fit(X_resampled, y_resampled)

# Output the best parameters and corresponding cross-validation accuracy
print("Best Parameters:", grid_tree.best_params_)
print("Best Cross-Validation Accuracy:", grid_tree.best_score_)

# Retrieve the best model from GridSearchCV
best_tree = grid_tree.best_estimator_

# Evaluate model performance on training set
y_train_pred = best_tree.predict(X_resampled)
train_accuracy = accuracy_score(y_resampled, y_train_pred)
print("\nTrain Accuracy Score:", train_accuracy)

# Process test set with the same preprocessor (without resampling)
X_test_processed = preprocessor.transform(X_test)

# Predict and evaluate on the test set
y_test_pred = best_tree.predict(X_test_processed)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("\nTest Accuracy Score:", test_accuracy)

# Confusion Matrix and Classification Report on test set
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))

# Optional: Additional metrics for better insight into class imbalance handling
from sklearn.metrics import precision_recall_fscore_support

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_test_pred, average='binary')
print("\nPrecision (Class 1):", precision)
print("Recall (Class 1):", recall)
print("F1-Score (Class 1):", fscore)


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'max_depth': 22, 'max_leaf_nodes': 100, 'min_samples_split': 50}
Best Cross-Validation Accuracy: 0.9223939657687936

Train Accuracy Score: 0.9304452332818643

Test Accuracy Score: 0.9010859610100745

Confusion Matrix:
 [[19663  1312]
 [  956   998]]

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.95     20975
           1       0.43      0.51      0.47      1954

    accuracy                           0.90     22929
   macro avg       0.69      0.72      0.71     22929
weighted avg       0.91      0.90      0.90     22929


Precision (Class 1): 0.43203463203463205
Recall (Class 1): 0.510747185261003
F1-Score (Class 1): 0.46810506566604126


In [14]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Retrieve the best estimator (the best decision tree model from Grid Search)
#best_tree = grid_tree.best_estimator_

# Set up the plot
plt.figure(figsize=(20, 10))  # Adjust the figure size for readability

# Plot the tree
plot_tree(
    best_tree,
    filled=True,                      # Colors the nodes by class
    feature_names=preprocessor.get_feature_names_out(),  # Use feature names from preprocessing
    class_names=['No Death', 'Death'] # Replace with appropriate class names
                     # Display only the top levels for readability; change as needed
)

# Show the plot
plt.show()

In [15]:
best_tree.get_depth()

19

In [16]:
import pandas as pd

# Retrieve the best estimator from GridSearchCV
#best_tree = grid_tree.best_estimator_

# Get feature importances from the best model
feature_importances = best_tree.feature_importances_

# Create a DataFrame to display feature names with their importance scores
feature_importance_df = pd.DataFrame({
    'Feature': preprocessor.get_feature_names_out(),
    'Importance': feature_importances
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

# Display the top features
print(feature_importance_df)

                                            Feature  Importance
0                     num__apache_4a_icu_death_prob    0.757473
1                num__apache_4a_hospital_death_prob    0.136397
2                          binary__intubated_apache    0.023999
3                         binary__diabetes_mellitus    0.013888
4                         binary__ventilated_apache    0.012875
5                          cat__ethnicity_Caucasian    0.008058
6                       cat__icu_admit_source_Floor    0.007434
7                     binary__apache_post_operative    0.006753
8             cat__apache_3j_bodysystem_Respiratory    0.005904
9                  cat__apache_3j_bodysystem_Sepsis    0.005337
10                                num__d1_sysbp_min    0.004179
11                                  num__d1_mbp_min    0.002460
12             cat__apache_2_bodysystem_Respiratory    0.002430
13                                 num__d1_spo2_min    0.002417
14                                 num__

In [17]:
import matplotlib.pyplot as plt

# Filter to include only features with importance greater than 0
important_features_df = feature_importance_df[feature_importance_df['Importance'] > 0]

# Plot the feature importances
plt.figure(figsize=(12, 8))
plt.barh(important_features_df['Feature'], important_features_df['Importance'], color='skyblue')
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title("Feature Importance in Best Decision Tree Model (Importance > 0)")
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()

In [18]:
# Predict probabilities and adjust threshold
y_test_prob = best_tree.predict_proba(X_test_processed)[:, 1]
threshold = 0.4
y_test_pred_adjusted = (y_test_prob >= threshold).astype(int)

# Evaluate the adjusted predictions
print("\nAdjusted Confusion Matrix (Test Data):\n", confusion_matrix(y_test, y_test_pred_adjusted))
print("\nAdjusted Classification Report (Test Data):\n", classification_report(y_test, y_test_pred_adjusted))


Adjusted Confusion Matrix (Test Data):
 [[19468  1507]
 [  863  1091]]

Adjusted Classification Report (Test Data):
               precision    recall  f1-score   support

           0       0.96      0.93      0.94     20975
           1       0.42      0.56      0.48      1954

    accuracy                           0.90     22929
   macro avg       0.69      0.74      0.71     22929
weighted avg       0.91      0.90      0.90     22929



### Random Forest Classifier

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

# Assuming X_resampled and y_resampled are the resampled data from SMOTE + Tomek Links
# Define the model
rf = RandomForestClassifier(random_state=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt']
}

# Define scoring metric focused on F1 for the minority class (class 1)
f1_minority = make_scorer(f1_score, pos_label=1)

In [20]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up the GridSearchCV
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=f1_minority,  # Focuses on F1 for class 1
    cv=cv,
    n_jobs=-1,  # Use all available cores
    verbose=2
)

In [21]:
grid_search.fit(X_resampled, y_resampled)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 20], 'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring=make_scorer(f1_score, response_method='predict', pos_label=1),
             verbose=2)

In [22]:
# Best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best F1 score for minority class: ", grid_search.best_score_)

# Predict on the original test set
best_rf = grid_search.best_estimator_
# Transform X_test using the same preprocessor pipeline
X_test_processed = preprocessor.transform(X_test)

# Now, use the transformed test set for prediction
y_pred = best_rf.predict(X_test_processed)

# Evaluate the performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

Best parameters found:  {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best F1 score for minority class:  0.9599268759671226
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     20975
           1       0.50      0.50      0.50      1954

    accuracy                           0.92     22929
   macro avg       0.73      0.73      0.73     22929
weighted avg       0.91      0.92      0.92     22929



In [23]:
# Evaluate model performance on training set
y_train_pred_rf = best_rf.predict(X_resampled)
train_accuracy_rf = accuracy_score(y_resampled, y_train_pred_rf)
print("\nTrain Accuracy Score:", train_accuracy_rf)

# Process test set with the same preprocessor (without resampling)
X_test_processed = preprocessor.transform(X_test)

# Predict and evaluate on the test set
y_test_pred_rf = best_rf.predict(X_test_processed)
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)
print("\nTest Accuracy Score:", test_accuracy_rf)


Train Accuracy Score: 0.9951926903424014

Test Accuracy Score: 0.9152601509006062


In [24]:
# Get feature importances from the best Random Forest model
feature_importances = best_rf.feature_importances_

# Create a DataFrame to display feature names with their importance scores
feature_importance_df = pd.DataFrame({
    'Feature': preprocessor.get_feature_names_out(),
    'Importance': feature_importances
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

# Display the top features
feature_importance_df

Feature  Importance
0                num__apache_4a_hospital_death_prob    0.127627
1                     num__apache_4a_icu_death_prob    0.121948
2                         binary__ventilated_apache    0.062849
3                          multi__gcs_verbal_apache    0.035942
4                            multi__gcs_eyes_apache    0.030565
5                          binary__intubated_apache    0.030091
6                           multi__gcs_motor_apache    0.028459
7                  cat__apache_3j_bodysystem_Sepsis    0.026266
8                                 num__d1_sysbp_min    0.022105
9                       cat__icu_admit_source_Floor    0.020620
10                       cat__icu_type_Med-Surg ICU    0.020417
11                                         num__age    0.016281
12  cat__icu_admit_source_Operating Room / Recovery    0.016242
13                                 num__d1_spo2_min    0.015947
14                        binary__diabetes_mellitus    0.015074
15                         binary__elective_surgery    0.013108
16                    binary__apache_post_operative    0.012992
17                         cat__ethnicity_Caucasian    0.012310
18                             num__d1_resprate_max    0.012161
19                                  num__d1_mbp_min    0.011662
20                             num__resprate_apache    0.011237
21                            num__d1_heartrate_max    0.011008
22                                 num__d1_temp_min    0.010953
23                               num__d1_diasbp_min    0.010832
24                                 num__h1_spo2_max    0.010183
25                            num__pre_icu_los_days    0.010158
26            cat__apache_3j_bodysystem_Respiratory    0.009766
27                             num__h1_resprate_max    0.009255
28                                 num__temp_apache    0.008664
29                                num__h1_sysbp_min    0.008640
30                             num__h1_resprate_min    0.008623
31             cat__apache_2_bodysystem_Respiratory    0.008498
32                                  num__h1_mbp_min    0.008180
33                                 num__h1_spo2_min    0.008119
34                            num__d1_potassium_max    0.007917
35                                    cat__gender_M    0.007576
36                                  num__map_apache    0.007520
37                                 num__d1_temp_max    0.007341
38                              num__d1_glucose_max    0.007332
39                            num__d1_heartrate_min    0.007159
40                                num__d1_sysbp_max    0.007015
41                             num__d1_resprate_min    0.007012
42                                         num__bmi    0.006657
43                            num__d1_potassium_min    0.006638
44                               num__h1_diasbp_min    0.006638
45                                      num__weight    0.006630
46                                num__h1_sysbp_max    0.006605
47                                      num__height    0.006448
48                              num__d1_glucose_min    0.006356
49                           num__heart_rate_apache    0.006317
50                            num__h1_heartrate_max    0.006266
51                                  num__d1_mbp_max    0.006063
52                                  num__h1_mbp_max    0.005971
53                            num__h1_heartrate_min    0.005888
54                               num__d1_diasbp_max    0.005763
55                               num__h1_diasbp_max    0.005458
56                                 num__d1_spo2_max    0.004754
57              cat__apache_3j_bodysystem_Metabolic    0.004589
58                               cat__icu_type_MICU    0.004562
59               cat__apache_2_bodysystem_Metabolic    0.004059
60           cat__apache_3j_bodysystem_Neurological    0.003881
61              cat__apache_2_bodysystem_Neurologic    0.003832
62          

In [25]:
# Filter to include only features with importance greater than 0
# Filter to include only features with importance greater than 0 and round the values

feature_importance_df['Importance'] = feature_importance_df['Importance'].round(2)

top_n = 25
top_features_df = feature_importance_df.head(top_n)
top_features_df

Feature  Importance
0                num__apache_4a_hospital_death_prob        0.13
1                     num__apache_4a_icu_death_prob        0.12
2                         binary__ventilated_apache        0.06
3                          multi__gcs_verbal_apache        0.04
4                            multi__gcs_eyes_apache        0.03
5                          binary__intubated_apache        0.03
6                           multi__gcs_motor_apache        0.03
7                  cat__apache_3j_bodysystem_Sepsis        0.03
8                                 num__d1_sysbp_min        0.02
9                       cat__icu_admit_source_Floor        0.02
10                       cat__icu_type_Med-Surg ICU        0.02
11                                         num__age        0.02
12  cat__icu_admit_source_Operating Room / Recovery        0.02
13                                 num__d1_spo2_min        0.02
14                        binary__diabetes_mellitus        0.02
15                         binary__elective_surgery        0.01
16                    binary__apache_post_operative        0.01
17                         cat__ethnicity_Caucasian        0.01
18                             num__d1_resprate_max        0.01
19                                  num__d1_mbp_min        0.01
20                             num__resprate_apache        0.01
21                            num__d1_heartrate_max        0.01
22                                 num__d1_temp_min        0.01
23                               num__d1_diasbp_min        0.01
24                                 num__h1_spo2_max        0.01

In [27]:
import matplotlib.pyplot as plt

# Plot the feature importances
plt.figure(figsize=(12, 8))
plt.barh(top_features_df['Feature'], top_features_df['Importance'], color='skyblue')
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title("Feature Importance in Random Forest (Importance > 0)")
plt.gca().invert_yaxis()  # Invert y-axis to have the most important feature at the top
plt.show()

In [28]:
top_features = [
    'num__apache_4a_hospital_death_prob',
    'num__apache_4a_icu_death_prob',
    'binary__ventilated_apache',
    'multi__gcs_verbal_apache',
    'multi__gcs_eyes_apache',
    'binary__intubated_apache',
    'multi__gcs_motor_apache',
    'cat__apache_3j_bodysystem_Sepsis',
    'num__d1_sysbp_min',
    'cat__icu_admit_source_Floor',
    'cat__icu_type_Med-Surg ICU',
    'num__age',
    'cat__icu_admit_source_Operating Room / Recovery',
    'num__d1_spo2_min',
    'binary__diabetes_mellitus',
    'binary__elective_surgery',
    'binary__apache_post_operative',
    'cat__ethnicity_Caucasian',
    'num__d1_resprate_max',
    'num__d1_mbp_min',
    'num__resprate_apache',
    'num__d1_heartrate_max',
    'num__d1_temp_min',
    'num__d1_diasbp_min',
    'num__h1_spo2_max'
]


In [29]:
# Convert X_resampled back to DataFrame and filter for the top 25 features
X_resampled_df = pd.DataFrame(X_resampled, columns=preprocessor.get_feature_names_out())
X_resampled_df = X_resampled_df[top_features]

# Step 4: Transform X_test without SMOTE and select only the top 25 features
X_test_processed = preprocessor.transform(X_test)
X_test_df = pd.DataFrame(X_test_processed, columns=preprocessor.get_feature_names_out())
X_test_df = X_test_df[top_features]

# Step 5: Set up Logistic Regression with GridSearchCV
log_reg = LogisticRegression(class_weight='balanced', random_state=42, max_iter=500)
param_grid = {
'C': [0.01, 0.1, 1, 10, 100],
'penalty': ['l1', 'l2'],
'solver': ['liblinear', 'saga'],
'class_weight': ['balanced', {0: 1, 1: 2}, {0: 1, 1: 3}]
}
grid_search = GridSearchCV(log_reg, param_grid, scoring='f1_weighted', cv=5, n_jobs=-1, verbose=2)

# Step 6: Fit GridSearchCV on the resampled training data with selected features
grid_search.fit(X_resampled_df, y_resampled)

# Step 7: Evaluate the best model on the test set
best_log_reg = grid_search.best_estimator_
y_test_pred = best_log_reg.predict(X_test_df)

print("Best parameters for Logistic Regression:", grid_search.best_params_)
print("\nClassification Report on Test Set:")
print(classification_report(y_test, y_test_pred))

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Model Accuracy on Test Set:", accuracy)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters for Logistic Regression: {'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}

Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.97      0.78      0.87     20975
           1       0.25      0.77      0.38      1954

    accuracy                           0.78     22929
   macro avg       0.61      0.78      0.62     22929
weighted avg       0.91      0.78      0.83     22929

Model Accuracy on Test Set: 0.7823280561734048


### KNN

In [30]:
from sklearn.neighbors import KNeighborsClassifier

# define function
knn = KNeighborsClassifier()
param_knn = { 'n_neighbors': range(3, 40, 3)}

# apply grid search:
grid_knn = GridSearchCV(knn, param_knn, cv = 5)
grid_knn.fit(X_resampled, y_resampled)

print(grid_knn.best_params_)
print(grid_knn.best_score_)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, classification_report

best_n_neighbors = grid_knn.best_params_['n_neighbors']

# Initialize and fit KNN with the best parameters
knn_best = KNeighborsClassifier(n_neighbors=best_n_neighbors)
knn_best.fit(X_resampled, y_resampled)

# Predict on the original test set
y_pred = knn_best.predict(X_test_processed)

# Calculate F1 scores
# Overall F1 score for both classes (averaged)
overall_f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' for imbalance
print("Overall F1 Score (weighted):", overall_f1)

# F1 score for each class
f1_class_0 = f1_score(y_test, y_pred, pos_label=0)
f1_class_1 = f1_score(y_test, y_pred, pos_label=1)
print("F1 Score for Class 0:", f1_class_0)
print("F1 Score for Class 1:", f1_class_1)

# Alternatively, print a full classification report to see precision, recall, and F1 for each class
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'n_neighbors': 6}
0.8823164559380441
Overall F1 Score (weighted): 0.8176473618841134
F1 Score for Class 0: 0.8618113665760367
F1 Score for Class 1: 0.34357366771159875

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.78      0.86     20975
           1       0.23      0.70      0.34      1954

    accuracy                           0.77     22929
   macro avg       0.60      0.74      0.60     22929
weighted avg       0.90      0.77      0.82     22929



In [31]:
# Evaluate model performance on training set
y_train_pred_knn = knn_best.predict(X_resampled)
train_accuracy_knn = accuracy_score(y_resampled, y_train_pred_knn)
print("\nTrain Accuracy Score:", train_accuracy_knn)

# Process test set with the same preprocessor (without resampling)
X_test_processed = preprocessor.transform(X_test)

# Predict and evaluate on the test set
y_test_pred_knn = knn_best.predict(X_test_processed)
test_accuracy_knn = accuracy_score(y_test, y_test_pred_knn)
print("\nTest Accuracy Score:", test_accuracy_knn)


Train Accuracy Score: 0.9157845306505786

Test Accuracy Score: 0.7716865105325134
